In [1]:
import numpy as np
from boxvectors import directions as directions
import Initial_Parameters as ip
from md import System
from md import md
from distribution import maxwellboltzmann
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.special import erfc
from scipy.constants import epsilon_0
%matplotlib inline

In [3]:
Symbols = ip.Symbols
Coefficients = ip.Coefficients
Charges = ip.Charges
N = ip.N*np.sum(Coefficients)
L = ip.L
T = ip.T
dt = ip.dt
p_rea = ip.p_rea
p = ip.p
std = ip.std
k_cut = ip.k_cut
k_max = ip.k_max_long_range
n_boxes_LJ = ip.n_boxes_LJ
n_boxes_coulomb = ip.n_boxes_short_range
Sys= System(Symbols, Coefficients, Charges, N/2)
Labels = Sys.get_Labels()
Sigma, Epsilon = Sys.get_LJ_parameter()
r_cut_coulomb = ip.r_cut_coulomb
r_switch = ip.r_switch
switch_parameter = ip.switch_parameter
m = Labels[:,0]

In [4]:
def get_random_starting_Positions(N,L):
    Positions = np.zeros((N,3))
    Positions[:,0] = np.linspace(0.1/N,L[0],N, endpoint = False)
    Positions[:,1] = np.linspace(0.1/N,L[1],N, endpoint = False)
    Positions[:,2] = np.linspace(0.1/N,L[2],N, endpoint = False)
    np.random.shuffle(Positions[:,0])
    np.random.shuffle(Positions[:,1])
    np.random.shuffle(Positions[:,2])
    return Positions
Positions = get_random_starting_Positions(N,L)
Velocities = maxwellboltzmann().sample_distribution(N,m,T)
Forces = np.zeros((N,3))
R = np.linalg.norm(Positions,axis=1)

In [5]:
MD = md(
    Positions, 
    R, 
    Labels, 
    Velocities,
    Forces, 
    L, 
    T, 
    std, 
    Sigma, 
    Epsilon, 
    switch_parameter,
    r_switch, 
    n_boxes_coulomb, 
    k_max, 
    dt, 
    p_rea,
    k_cut,
    r_cut_coulomb)


In [6]:
MD.forces = MD.get_forces()

In [8]:
print MD.get_energy()
print MD.get_potential()
print MD.forces

-8.07742261715e-26
[  2.60127722e-08   2.64001889e-08   4.96370754e-08   1.79501964e-08
   4.65098903e-08   4.13047149e-08   4.59768959e-09  -1.25148525e-08
  -5.18033085e-09   3.63327790e-08  -1.47016773e-08  -2.64254123e-08
  -3.74886483e-08   2.54594175e-09  -5.79888137e-08   8.69635040e-09
  -6.49183962e-08   2.87470007e-08  -5.20527567e-08  -1.51004418e-08]
[[  1.19072764e-25   2.50428023e-26   1.35516510e-26]
 [  3.69869115e-26  -8.44915867e-26   7.41521148e-26]
 [  6.11815657e-26   1.04878046e-25  -7.58902650e-26]
 [ -7.99577669e-26  -2.46430775e-25   2.86552436e-26]
 [  2.26243988e-25  -4.31284846e-26   2.28412089e-26]
 [ -1.31793100e-25   9.15622651e-26   8.37231248e-26]
 [ -1.30996740e-25   2.90002463e-25  -1.22512827e-25]
 [  7.54503160e-26  -4.68816676e-26   6.83627692e-26]
 [ -3.08468099e-25   2.00041063e-25   2.40356150e-25]
 [ -5.72828834e-26  -9.58310472e-26  -1.74361617e-25]
 [  1.21877109e-25  -2.44948221e-25  -1.04024424e-25]
 [ -1.65684994e-25   1.34527127e-25   8.1

In [ ]:
Temperature = np.zeros(10)
for i in np.arange(10):
    Positions_New, Velocities_New, Forces_New = MD.propagte_system()
    MD.positions = Positions_New
    MD.velocities = Velocities_New
    MD.forces = Forces_New
    Temperature[i] = MD.get_Temperature()
plt.plot(Temperature)